In [2]:
from tkinter import *
from PIL import Image
import numpy as np

In [34]:
def read_input():
    root = Tk()
    root.title("Draw the digit")
    root.resizable(0,0)
    down = False
    def press(event):
        global down
        down = True
    
    def release(event):
        global down
        down = False

    def motion(event):
        if down:
            c.create_oval(event.x, event.y, event.x+1, event.y+1, width=8, fill="black")

    c = Canvas(root, bg="white", width=400, height= 400)
    c.configure(cursor="crosshair")
    c.pack()

    c.bind("<Button-1>", press)
    c.bind("<ButtonRelease-1>", release)

    c.bind('<Motion>', motion)

    def btn_callback():
        global c
        c.postscript(file="digit.eps")
        img = Image.open("digit.eps")
        img.save("digit.png", "png")
        img = im.convert('1')      # Convert to black&white
        size = 28, 28
        img.thumbnail(size, Image.ANTIALIAS)
        arr = np.array(im)
        img.save("small_digit.png", "png")
        resh = arr.reshape((28*28, 1))
        print(resh.shape)
        
        
    b = Button(root, text="OK", command=btn_callback)
    b.pack()

    root.mainloop()

In [ ]:
root = Tk()
root.title("Draw the digit")
root.resizable(0,0)
down = False
def press(event):
    global down
    down = True

def release(event):
    global down
    down = False

def motion(event):
    if down:
        c.create_oval(event.x, event.y, event.x+1, event.y+1, width=8, fill="black")

c = Canvas(root, bg="white", width=400, height= 400)
c.configure(cursor="crosshair")
c.pack()

c.bind("<Button-1>", press)
c.bind("<ButtonRelease-1>", release)

c.bind('<Motion>', motion)

def btn_callback():
    global c
    c.postscript(file="digit.eps")
    img = Image.open("digit.eps")
    img.save("digit.png", "png")
    img = im.convert('1')      # Convert to black&white
    size = 28, 28
    img.thumbnail(size, Image.ANTIALIAS)
    arr = np.array(im)
    img.save("small_digit.png", "png")
    resh = arr.reshape((28*28, 1))
    print(resh.shape)


b = Button(root, text="OK", command=btn_callback)
b.pack()

root.mainloop()

In [35]:
read_input()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kirill/anaconda3/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "<ipython-input-34-23d4a0c037e9>", line 29, in btn_callback
    c.postscript(file="digit.eps")
  File "/home/kirill/anaconda3/lib/python3.5/tkinter/__init__.py", line 2438, in postscript
    self._options(cnf, kw))
_tkinter.TclError: invalid command name ".140190753689896"
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kirill/anaconda3/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "<ipython-input-34-23d4a0c037e9>", line 29, in btn_callback
    c.postscript(file="digit.eps")
  File "/home/kirill/anaconda3/lib/python3.5/tkinter/__init__.py", line 2438, in postscript
    self._options(cnf, kw))
_tkinter.TclError: invalid command name ".140190753689896"


In [10]:
im = Image.open("digit.png")
im = im.convert('1')      # Convert to black&white
A = np.array(im)             # Creates an array, white pixels==True and black pixels==False
new_A=np.empty((A.shape[0],A.shape[1]),None) 

In [22]:
r = im.getpixel((1, 1))
q = im.getpixel((100, 100))
r, q

(0, 255)

In [23]:
im.size

(452, 452)

In [26]:
pix = im.load()
pix[100, 100]

255

In [29]:
size = 28, 28
outfile =  "small_digit.png"
im = Image.open("digit.png")
im = im.convert('1')      # Convert to black&white
im.thumbnail(size, Image.ANTIALIAS)
arr = np.array(im)
im.save(outfile, "png")

In [31]:
arr.shape

(28, 28)

In [32]:
resh = arr.reshape((28*28, 1))
resh.shape

(784, 1)